In [1]:
from ngsolve import *
import netgen.gui
%gui tk

from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.05))
mesh.Curve(3)
Draw (mesh)

In [2]:
def SolveStokes(X):
    ux,uy,p = X.TrialFunction()
    vx,vy,q = X.TestFunction()

    div_u = grad(ux)[0]+grad(uy)[1]
    div_v = grad(vx)[0]+grad(vy)[1]

    a = BilinearForm(X)
    a += SymbolicBFI(grad(ux)*grad(vx)+grad(uy)*grad(vy) - div_u*q - div_v*p)
    a.Assemble()

    gfu = GridFunction(X)
    uin = 1.5*4*y*(0.41-y)/(0.41*0.41)
    gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
    gfu.vec.data += inv * res
    return gfu
 

In [3]:
def TestSpace(X):
    gfu = SolveStokes(X)
    velocity = CoefficientFunction(gfu.components[0:2])
    pressure = CoefficientFunction(gfu.components[2])
    Draw(pressure, mesh, "press")
    Draw(velocity, mesh, "vel") 

In [4]:
V = H1(mesh, order=2, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=1)
X = FESpace([V,V,Q])
#TestSpace(X) # why doesn't this work? The velocity field is messed up!
# Maybe because pvelocity and pressure go out of scope when the function
# returns -- that may reset their memory to garbage...
try:
    gfu = SolveStokes(X)
except RuntimeError as a:
    print("test1: ", a)

velocity = CoefficientFunction(gfu.components[0:2])
pressure = CoefficientFunction(gfu.components[2])
Draw(pressure, mesh, "press")
Draw(velocity, mesh, "vel") 

In [5]:
V = H1(mesh, order=4, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=3)
X = FESpace([V,V,Q])

try:
    gfu = SolveStokes(X)
except RuntimeError as a:
    print("test2: ",a)
    
velocity = CoefficientFunction(gfu.components[0:2])
pressure = CoefficientFunction(gfu.components[2])
Draw(pressure, mesh, "press")
Draw(velocity, mesh, "vel") 

In [6]:
V = H1(mesh, order=2, dirichlet="wall|inlet|cyl")
Q = L2(mesh, order=1)
print ("V.ndof =", V.ndof, ", Q.ndof =", Q.ndof)
X = FESpace([V,V,Q])

try:
    gfu = SolveStokes(X)
except RuntimeError as a:
    print("test3: ",a)

velocity = CoefficientFunction(gfu.components[0:2])
pressure = CoefficientFunction(gfu.components[2])
Draw(pressure, mesh, "press")
Draw(velocity, mesh, "vel")


V.ndof = 1690 , Q.ndof = 2364
test3:  UmfpackInverse: Numeric factorization failed.


In [7]:
V = H1(mesh, order=2, dirichlet="wall|inlet|cyl")
V.order[TRIG]=3
print ("V.ndof =", V.ndof, ", Q.ndof =", Q.ndof)
Q = L2(mesh, order=1)
X = FESpace([V,V,Q])

try:
    gfu = SolveStokes(X)
except RuntimeError as a:
    print("test4: ",a)

velocity = CoefficientFunction(gfu.components[0:2])
pressure = CoefficientFunction(gfu.components[2])
Draw(pressure, mesh, "press")
Draw(velocity, mesh, "vel")


V.ndof = 2478 , Q.ndof = 2364


In [11]:
# this one returns no error, but the pressure looks very weird!
V = H1(mesh, order=1, dirichlet="wall|inlet|cyl")
V.order[TRIG]=3
Q = H1(mesh, order=1)
X = FESpace([V,V,Q])

try:
    gfu = SolveStokes(X)
except RuntimeError as a:
    print("test5: ",a)

velocity = CoefficientFunction(gfu.components[0:2])
pressure = CoefficientFunction(gfu.components[2])
Draw(pressure, mesh, "press")
Draw(velocity, mesh, "vel")


In [13]:
V = FESpace("VectorH1", mesh, order=3, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=2)
X = FESpace([V,Q])

u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += SymbolicBFI(InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)
a.Assemble()

gfu = GridFunction(X)
uin = 1.5*4*y*(0.41-y)/(0.41*0.41)
gfu.components[0].components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
gfu.vec.data += inv * res
velocity = CoefficientFunction(gfu.components[0])
pressure = CoefficientFunction(gfu.components[1])
Draw(pressure, mesh, "press")
Draw(velocity, mesh, "vel")
